## OLP_LLM: Pipeline for generating FOON graphs via LLM Prompting

## Step 1 :- Generate a FOON via LLM Prompting

1. Initialize libraries needed for FOON (``FOON_graph_analyser.py``) as well as OpenAI api.

2. Perform 2-stage prompting for recipe prototype.
    - In the first stage, we ask the LLM for a *high-level recipe* (list of instructions) and a *list of objects* needed for completing the recipe.
    - In the second stage, we ask the LLM for a breakdown of *state changes* that happen for each step of the recipe; specifically, we ask for the *preconditions* and *effects* of each action, which is similar to how a functional unit in FOON has *input* and *output object nodes*.

### Initialize library

In [1]:
%load_ext autoreload
%autoreload 2

from olp_library import *
# import FOON_graph_analyser as fga

### Language to  OLP

In [2]:
models = [
    "gpt-3.5-turbo",
    "gpt-4",
    ]

in_context_file = "incontext_examples.txt"
user_task = "How do I make coffee?"
user_task = "How do I make a Bloody Mary cocktail?"
user_task = "How do I make a Greek salad?"

# user_task = "How do I make a burger?"
# user_task = "How do I unlock a door?"
# user_task = "How do I turn on the lights in my house"

high_level_plan, object_level_plan, plan_objects = generate_olp(user_task, in_context_file, models[1], verbose=True)

print(plan_objects)

*************************************************************************
Stage1 Prompting
*************************************************************************
Model: gpt-4
Complete prompt:
[
    {
        "role": "system",
        "content": "You are an LLM that understands how to generate concise high level plans for arbitrary tasks involving objects. Only focus on what happens to objects.  Stay consistent with object names and use one verb per step. Assume you have all objects necessary for the plan and do not need to obtain anything."
    },
    {
        "role": "user",
        "content": "How do I make a Greek salad?\nAfter listing the high level plan, list all the unique objects used in the high level plan, including the object created after solving the task, ignoring state changes to those objects. Follow the format 'unique_objects:object_1, object_2, object_3, ...'"
    }
]
*************************************************************************
Stage 1 Response: 
1. Cle

In [3]:
## Visualize Outputs
print("***********************\n   High level plan\n***********************\n",   high_level_plan,   "\n")
print("***********************\n   Object level plan\n***********************")
print(*object_level_plan, sep="\n")
print("\n***********************\n   Plan objects\n***********************\n",      plan_objects,      "\n")

***********************
   High level plan
***********************
 1. Clean lettuce.
2. Chop lettuce.
3. Place chopped lettuce in bowl.
4. Slice tomatoes.
5. Add sliced tomatoes to bowl.
6. Slice cucumbers.
7. Add sliced cucumbers to bowl.
8. Slice onions.
9. Add sliced onions to bowl.
10. Slice olives.
11. Add sliced olives to bowl.
12. Crumble feta cheese.
13. Add crumbled feta cheese to bowl.
14. Drizzle olive oil over salad in bowl.
15. Sprinkle oregano over salad in bowl.
16. Toss salad in bowl.
17. Serve salad.

unique_objects: lettuce, tomatoes, cucumbers, onions, olives, feta cheese, olive oil, oregano, bowl, salad 

***********************
   Object level plan
***********************
{'Step': 'Clean lettuce', 'Objects': ['lettuce'], 'Action': 'Clean', 'StateChanges': {'lettuce': {'Precondition': ['dirty'], 'Effect': ['clean']}}}
{'Step': 'Chop lettuce', 'Objects': ['lettuce'], 'Action': 'Chop', 'StateChanges': {'lettuce': {'Precondition': ['whole'], 'Effect': ['chopped']}}}
{

In [4]:
## Test Foon creation
plan_step = object_level_plan[0]

sample_unit = create_olp_functionalUnit(plan_step,plan_objects)
sample_unit.print_functions[2](version=1)

Creating functional unit for: Clean lettuce
Objects: ['lettuce']
Current object is : lettuce
	 lettuce state changes: {'Precondition': ['dirty'], 'Effect': ['clean']}
		 Precondition : dirty || related objects: []
		 Effect : clean || related objects: []
O	lettuce
S	dirty
M	Clean	<Assumed>
O	lettuce
S	clean


In [5]:
#Complete FOON prototype:
FOON_prototype = []

for step in object_level_plan:
    print(step['Step'])
    # -- now we will create functional units that follow the FOON format:
    new_unit = create_olp_functionalUnit(step, plan_objects)      
    new_unit.print_functions[2]()
    
    # input()
    print()

    # -- add the functional unit to the FOON prototype:
    FOON_prototype.append(new_unit)

Clean lettuce
Creating functional unit for: Clean lettuce
Objects: ['lettuce']
Current object is : lettuce
	 lettuce state changes: {'Precondition': ['dirty'], 'Effect': ['clean']}
		 Precondition : dirty || related objects: []
		 Effect : clean || related objects: []
O	lettuce
S	dirty
M	Clean	<Assumed>
O	lettuce
S	clean

Chop lettuce
Creating functional unit for: Chop lettuce
Objects: ['lettuce']
Current object is : lettuce
	 lettuce state changes: {'Precondition': ['whole'], 'Effect': ['chopped']}
		 Precondition : whole || related objects: []
		 Effect : chopped || related objects: []
O	lettuce
S	whole
M	Chop	<Assumed>
O	lettuce
S	chopped

Place chopped lettuce in bowl
Creating functional unit for: Place chopped lettuce in bowl
Objects: ['lettuce', 'bowl']
Current object is : lettuce
	 lettuce state changes: {'Precondition': ['not in bowl'], 'Effect': ['in bowl']}
		 Precondition : not in bowl || related objects: ['bowl']
		 Effect : in bowl || related objects: ['bowl']
Current obje

### Writing FOON to a Text File

In [6]:
# -- save the prototype FOON graph as a text file, which we will then run with a parser to correct numbering:
if not os.path.exists('preprocess/'):
	os.makedirs('preprocess/')

if not os.path.exists('postprocess/'):
	os.makedirs('postprocess/')

file_ = open('preprocess/prototype.txt', 'w')
file_.write('#\tFOON Prototype\n#\t-- Task Prompt: {0}\n//\n'.format(user_task))
for unit in FOON_prototype:
	file_.write(unit.getFunctionalUnitText())
	unit.print_functions[2]()
	print('//')

file_.close()

O	lettuce
S	dirty
M	Clean	<Assumed>
O	lettuce
S	clean
//
O	lettuce
S	whole
M	Chop	<Assumed>
O	lettuce
S	chopped
//
O	lettuce
S	not in 	 [bowl]
O	bowl
S	empty
M	Place	<Assumed>
O	lettuce
S	in 	 [bowl]
O	bowl
S	contains 	{lettuce}
//
O	tomatoes
S	whole
M	Slice	<Assumed>
O	tomatoes
S	sliced
//
O	tomatoes
S	not in 	 [bowl]
O	bowl
S	contains 	{lettuce}
M	Add	<Assumed>
O	tomatoes
S	in 	 [bowl]
O	bowl
S	contains 	{lettuce,tomatoes}
//
O	cucumbers
S	whole
M	Slice	<Assumed>
O	cucumbers
S	sliced
//
O	cucumbers
S	not in 	 [bowl]
O	bowl
S	contains 	{lettuce,tomatoes}
M	Add	<Assumed>
O	cucumbers
S	in 	 [bowl]
O	bowl
S	contains 	{lettuce,tomatoes,cucumbers}
//
O	onions
S	whole
M	Slice	<Assumed>
O	onions
S	sliced
//
O	onions
S	not in 	 [bowl]
O	bowl
S	contains 	{lettuce,tomatoes,cucumbers}
M	Add	<Assumed>
O	onions
S	in 	 [bowl]
O	bowl
S	contains 	{lettuce,tomatoes,cucumbers,onions}
//
O	olives
S	whole
M	Slice	<Assumed>
O	olives
S	sliced
//
O	olives
S	not in 	 [bowl]
O	bowl
S	contains 	{lettuce,tomato

## Step 2 :- FOON to PDDL
1. Parse FOON file -- this step is important to ensure that all labels are unique and that the generated file follows the FOON syntax.

2. (Optional) Visualize FOON graph

3. Run ``FOON_to_PDDL.py`` script to generate FOON macro-operators

### Parse and clean generated FOON

In [7]:
# -- running parsing module to ensure that FOON labels and IDs are made consistent for further use:
#		(it is important that each object and state type have a *UNIQUE* identifier)
fpa.skip_JSON_conversion = True		# -- we don't need JSON versions of a FOON
fpa.skip_index_check = True			# -- always create a new set of index files

fpa.source_dir = './preprocess/'	
fpa.target_dir = './postprocess/'
fpa._run_parser()

-- [FOON_parser] : Initiating parsing procedure!


 -- [FOON_parser] : Commencing parsing...
  -- parsing 'prototype.txt'...

 -- [FOON_parser] : Saving corrected files to './postprocess/'...
  -- Saving 'prototype.txt'...

-- Revising object label senses using WordNet and Concept-Net...


SUMMARY OF CHANGES:
  -- new total of OBJECTS : 10
  -- new total of STATES : 13
  -- new total of MOTIONS : 10

 -- [FOON_parser] : Parsing complete!


### Run FOON visualization tool

In [8]:
# -- after running the parser, take the 'prototype.txt' file and plot it using the following tool: https://davidpaulius.github.io/foon-view/
fga._startFOONview()